In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler as SS
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.model_selection import GridSearchCV
from sklearn.cluster import KMeans

In [3]:
df = pd.read_excel('database_cluster.xlsx')

In [11]:
df['Segmentos'].value_counts()

Campeões                  205
Potencial cliente fiel    181
Promissores               125
Clientes recentes          75
Requer atenção             67
Perdidos                   44
Hibernando                 43
Clientes fiéis             28
Prestes a dormir           24
Em risco                   17
Não pode perdê-los         15
Name: Segmentos, dtype: int64

In [12]:
segmentos = {'Campeões': '11', 'Requer atenção': '10', 'Promissores': '9', 'Clientes fiéis': '8',
        'Potencial cliente fiel': '7', 'Clientes recentes': '6', 'Em risco': '5', 'Hibernando': '4',
        'Prestes a dormir': '3', 'Prestes a dormir': '2', 'Perdidos': '1', 'Não pode perdê-los': '0'}
df['Classificação'] = df['Segmentos'].map(segmentos)

In [13]:
df

,Rede,Recência,Frequência,Valor,Segmentos,Classificação
0,DROGASIL - SP,7,443,3.888629e+06,Campeões,11
1,REDE SANTA CRUZ - SP,7,410,1.763372e+06,Campeões,11
2,EMPREENDIMENTOS PAGUE MENOS - CE,7,432,1.570881e+06,Campeões,11
3,D CENTER DISTRIBUIDORA LTDA - GO,7,338,1.510718e+06,Campeões,11
4,DROG SAO PAULO/PACHECO SA - GO,7,373,1.484749e+06,Campeões,11
...,...,...,...,...,...,...
819,EMMARKA/PLUSFARMA - PE,62,1,1.404000e+03,Perdidos,1
820,DISTRIPHARMA DISTR DE MEDIC LTDA - RS,62,1,7.020000e+02,Perdidos,1
821,ALTHIS COM DE MEDIC E MAT MEDICO - SC,60,5,3.610000e+02,Perdidos,1
822,CIAMED DIST DE MED LTDA - RS,60,2,2.750000e+02,Perdidos,1


In [14]:
X = df.drop(['Rede', 'Segmentos', 'Classificação'], axis = 1)
X


,Recência,Frequência,Valor
0,7,443,3.888629e+06
1,7,410,1.763372e+06
2,7,432,1.570881e+06
3,7,338,1.510718e+06
4,7,373,1.484749e+06
...,...,...,...
819,62,1,1.404000e+03
820,62,1,7.020000e+02
821,60,5,3.610000e+02
822,60,2,2.750000e+02


In [15]:
y = df[df.columns[-1:]]
y

,Classificação
0,11
1,11
2,11
3,11
4,11
...,...
819,1
820,1
821,1
822,1


In [16]:
#Separa-se as bases teste e treino
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3)

In [17]:
#Ultilizamos o GridSearchCV com varios parametros para nos indicar o melhor parmetro para se ultilizar o RandomForestClassifier.

parameters ={'n_estimators' : (10,30,50,70,90,100)

              , 'criterion' : ('gini', 'entropy')

              , 'max_depth' : (3,5,7,9,10)

              , 'max_features' : ('auto', 'sqrt')

              , 'min_samples_split' : (2,4,6)

              , 'min_weight_fraction_leaf' : (0.0,0.1,0.2,0.3)

             }

RF_grid  = GridSearchCV(RandomForestClassifier(n_jobs = -1, oob_score= False), param_grid = parameters, cv = 3, verbose = True)

RF_grid_model = RF_grid.fit(X_train, y_train)

RF_grid_model.best_estimator_

Fitting 3 folds for each of 1440 candidates, totalling 4320 fits


C:\ProgramData\anaconda3\envs\ds-3.8\lib\site-packages\sklearn\model_selection\_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\ProgramData\anaconda3\envs\ds-3.8\lib\site-packages\sklearn\ensemble\_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
C:\ProgramData\anaconda3\envs\ds-3.8\lib\site-packages\sklearn\model_selection\_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\ProgramData\anaconda3\envs\d

RandomForestClassifier(max_depth=10, n_estimators=70, n_jobs=-1)

In [26]:
#Aplicamos os parametros apontado no GridSearchCV no RandomForestClassifier é medimos a acuracia do modelo (aproximadamenteo 80%)

modelo = RF_grid_model.best_estimator_

modelo.fit(X_train,y_train)

prediction = modelo.predict(X_test)

print(metrics.accuracy_score(y_test,prediction))

0.9516129032258065


C:\Users\diullio.santos\AppData\Local\Temp\ipykernel_19768\2009620979.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  modelo.fit(X_train,y_train)


In [27]:
from sklearn.metrics import confusion_matrix

In [28]:
confusion = confusion_matrix(y_test, prediction)
confusion

array([[ 2,  0,  0,  0,  0,  0,  1,  0,  0,  0,  0],
       [ 0, 17,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0, 15,  0,  0,  0,  0,  0,  1,  0,  0],
       [ 0,  0,  0, 65,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  4,  0,  0,  3,  0,  0,  0],
       [ 1,  1,  0,  0,  0, 14,  1,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  4,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0, 15,  0,  0,  1],
       [ 0,  0,  0,  0,  1,  0,  0,  0, 54,  0,  1],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  1,  3,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 43]], dtype=int64)

In [40]:
classes = ['Campeões','Potencial cliente fiel','Promissores','Clientes recentes','Requer atenção','Perdidos','Hibernando','Clientes fiéis','Prestes a dormir','Em risco','Não pode perdê-los']

In [41]:
from sklearn.metrics import classification_report

In [45]:
# Cálculo da acurácia
accuracy = np.sum(np.diag(confusion)) / np.sum(confusion)

# Cálculo da precisão para cada classe
precision_scores = np.diag(confusion) / np.sum(confusion, axis=0)

# Cálculo do Recall para cada classe
recall_scores = np.diag(confusion) / np.sum(confusion, axis=1)

# Cálculo do F1-Score para cada classe
f1_scores = 2 * (precision_scores * recall_scores) / (precision_scores + recall_scores)

# Criação do relatório de classificação
classification_report_str = classification_report(np.arange(len(classes)), np.arange(len(classes)),
                                                  target_names=classes,
                                                  labels=np.arange(len(classes)),
                                                  sample_weight=None,
                                                  digits=2,
                                                  output_dict=False,
                                                  zero_division=0)

# Imprimir os resultados
print('Acurácia:', accuracy)
print('\nPrecisão:')
for i, class_name in enumerate(classes):
    print(class_name, ':', precision_scores[i])
print('\nRecall (Revocação):')
for i, class_name in enumerate(classes):
    print(class_name, ':', recall_scores[i])
print('\nF1-Score:')
for i, class_name in enumerate(classes):
    print(class_name, ':', f1_scores[i])

# Imprimir o relatório de classificação completo
print('\nRelatório de Classificação:')
print(classification_report_str)

Acurácia: 0.9516129032258065

Precisão:
Campeões : 0.6666666666666666
Potencial cliente fiel : 0.9444444444444444
Promissores : 1.0
Clientes recentes : 1.0
Requer atenção : 0.8
Perdidos : 1.0
Hibernando : 0.6666666666666666
Clientes fiéis : 0.8333333333333334
Prestes a dormir : 0.9642857142857143
Em risco : 1.0
Não pode perdê-los : 0.9555555555555556

Recall (Revocação):
Campeões : 0.6666666666666666
Potencial cliente fiel : 1.0
Promissores : 0.9375
Clientes recentes : 1.0
Requer atenção : 0.5714285714285714
Perdidos : 0.8235294117647058
Hibernando : 1.0
Clientes fiéis : 0.9375
Prestes a dormir : 0.9642857142857143
Em risco : 0.75
Não pode perdê-los : 1.0

F1-Score:
Campeões : 0.6666666666666666
Potencial cliente fiel : 0.9714285714285714
Promissores : 0.967741935483871
Clientes recentes : 1.0
Requer atenção : 0.6666666666666666
Perdidos : 0.9032258064516129
Hibernando : 0.8
Clientes fiéis : 0.8823529411764706
Prestes a dormir : 0.9642857142857143
Em risco : 0.8571428571428571
Não pode